In [1]:
import sys
from pathlib import Path
sys.path.append(str(Path("..").resolve() / "src"))

# Datebase Tests

In [1]:
from sqlalchemy import text
from market_extractor.db.engine import make_engine
from market_extractor.db.check import get_tables, delete_table, get_table_columns, get_table
from market_extractor.db.create import create_market, create_ticker

engine = make_engine()

In [3]:
get_tables()

['core.listing', 'core.market', 'core.symbol', 'core.ticker']

In [4]:
get_table('core.ticker')

[{'ticker_id': 1,
  'market_id': 1,
  'name': 'Tesla',
  'symbol': 'TSLA',
  'asset': 'equity',
  'sector': 'utilities',
  'industry': 'other'}]

In [3]:
create_ticker(
    engine, 
    market_id = 1, 
    name = "Tesla", 
    symbol = "TSLA", 
    asset = "equity", 
    sector = "utilities", 
    industry = "other"
    )

1

In [ ]:
create_market(
    engine, 
    name="Stockholm Stock Exchange", 
    code="OMX", 
    country="SE", 
    currency="SEK", 
    timezone="Europe/Stockholm"
    )

In [ ]:
get_table_columns('core.market')

In [ ]:
delete_table('raw.ohlcv')

In [ ]:
with engine.connect() as conn:
    result = conn.execute(text("SELECT now() AS db_time;")).mappings().one()
result

In [ ]:
# Insert one candle from the notebook

sql = text("""
INSERT INTO raw.ohlcv(listing_id, tf, ts, open, high, low, close, volume, source)
SELECT l.listing_id, '1d', '2026-02-26T00:00:00Z'::timestamptz, 200, 210, 190, 205, 999, 'notebook'
FROM core.listing l
JOIN core.market m ON m.market_id = l.market_id
WHERE m.code = 'NASDAQ' AND l.local_ticker = 'AAPL'
ON CONFLICT (listing_id, tf, ts) DO NOTHING;
""")

with engine.begin() as conn:
    conn.execute(sql)

"Inserted (or already existed)."

In [ ]:
# And verify

from sqlalchemy import text
from market_extractor.db.engine import make_engine

engine = make_engine()

q = text("""
SELECT o.ts, o.open, o.close, o.volume, o.source
FROM raw.ohlcv o
JOIN core.listing l ON l.listing_id = o.listing_id
JOIN core.market m ON m.market_id = l.market_id
WHERE m.code='NASDAQ' AND l.local_ticker='AAPL' AND o.tf='1d'
ORDER BY o.ts DESC
LIMIT 5;
""")

with engine.connect() as conn:
    rows = conn.execute(q).mappings().all()

rows

## Get listing ID

In [ ]:
from market_extractor.db.engine import make_engine
import market_extractor.db.lookup as lookup

import inspect
print(inspect.getsource(lookup.get_or_create_symbol))
lookup.__file__

engine = make_engine()

ref = lookup.get_or_create_listing(engine, market_code="NASDAQ", symbol_code="AAPL")
ref